# Feature Engineering: Application

In [38]:
from pathlib import Path
import re
import time
from datetime import datetime
from dateutil import relativedelta

import numpy as np
import pandas as pd
import toad

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import config
from util.db_helper import SQLite
from util.data_helper import Data
from projects.home_credit.features.ft_app import FTApp

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
db_path = Path(config.ROOT_DIR, 'data', 'home_credit_default_risk.db')
SQLite.get_conn(db_path)

In [26]:
sql = """
    SELECT * 
    FROM application_train
"""
df_app = SQLite.query(sql)
Data.dump('df_app', df_app)

In [27]:
toad.detect(df_app)

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
SK_ID_CURR,int64,307511,0.00%,307511,278180.518577,102790.175348,100002.0,103590.2,135692.0,278202.0,367142.5,420729.0,452712.9,456255.0
TARGET,int64,307511,0.00%,2,0.080729,0.272419,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
NAME_CONTRACT_TYPE,object,307511,0.00%,2,Cash loans:90.48%,Revolving loans:9.52%,None,None,None,None,None,None,Cash loans:90.48%,Revolving loans:9.52%
CODE_GENDER,object,307511,0.00%,3,F:65.83%,M:34.16%,XNA:0.00%,None,None,None,None,F:65.83%,M:34.16%,XNA:0.00%
FLAG_OWN_CAR,object,307511,0.00%,2,N:65.99%,Y:34.01%,None,None,None,None,None,None,N:65.99%,Y:34.01%
FLAG_OWN_REALTY,object,307511,0.00%,2,Y:69.37%,N:30.63%,None,None,None,None,None,None,Y:69.37%,N:30.63%
CNT_CHILDREN,int64,307511,0.00%,15,0.417052,0.722121,0.0,0.0,0.0,0.0,1.0,2.0,3.0,19.0
AMT_INCOME_TOTAL,float64,307511,0.00%,2548,168797.919297,237123.146279,25650.0,45000.0,81000.0,147150.0,202500.0,270000.0,472500.0,117000000.0
AMT_CREDIT,float64,307511,0.00%,5603,599025.999706,402490.776996,45000.0,76410.0,180000.0,513531.0,808650.0,1133748.0,1854000.0,4050000.0
AMT_ANNUITY,float64,307511,0.00%,13672,27108.573909,14493.737315,1615.5,6182.91,11074.5,24903.0,34596.0,45954.0,70006.5,258025.5


In [46]:
df_app = Data.load('df_app')

In [47]:
df_dim = pd.DataFrame({'SK_ID_CURR': df_app['SK_ID_CURR'].unique()})

In [48]:
df_ft = FTApp.get_features(df_dim, df_app)

/Users/yuanzhenyu/workspace/services/ds-fintech/projects/home_credit/features/ft_app.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if c == 'SK_ID_CURR':


In [49]:
df_ft

,SK_ID_CURR,ft_app_young_age,ft_app_id_renewal_years,ft_app_annuity_income_pct,ft_app_credit_income_ratio,ft_app_credit_annuity_ratio,ft_app_credit_goods_ratio,ft_app_children_ratio,ft_app_income_per_child,ft_app_income_per_person,ft_app_days_employed_pct,ft_app_phone_chg_employ_ratio,ft_app_short_employment,ft_app_code_gender,ft_app_weekday_appr_process_start,ft_app_hour_appr_process_start,ft_app_flag_mobil,ft_app_flag_cont_mobile,ft_app_flag_email,ft_app_flag_phone,ft_app_flag_work_phone,ft_app_flag_emp_phone,ft_app_flag_document_3,ft_app_flag_document_4,ft_app_flag_document_5,ft_app_flag_document_6,ft_app_flag_document_7,ft_app_flag_document_8,ft_app_flag_document_9,ft_app_flag_document_11,ft_app_flag_document_18,ft_app_flag_own_car,ft_app_flag_own_realty,ft_app_occupation_type,ft_app_organization_type,ft_app_name_income_type,ft_app_name_contract_type,ft_app_name_education_type,ft_app_name_family_status,ft_app_name_housing_type,ft_app_name_type_suite,ft_app_live_city_not_work_city,ft_app_live_region_not_work_region,ft_app_reg_city_not_live_city,ft_app_reg_city_not_work_city,ft_app_reg_region_not_live_region,ft_app_reg_region_not_work_region,ft_app_housetype_mode,ft_app_emergencystate_mode,ft_app_fondkapremont_mode,ft_app_wallsmaterial_mode,ft_app_amt_annuity,ft_app_amt_credit,ft_app_amt_income_total,ft_app_amt_req_credit_bureau_hour,ft_app_amt_req_credit_bureau_day,ft_app_amt_req_credit_bureau_week,ft_app_amt_req_credit_bureau_mon,ft_app_amt_req_credit_bureau_qrt,ft_app_amt_req_credit_bureau_year,ft_app_apartments_avg,ft_app_basementarea_avg,ft_app_commonarea_avg,ft_app_elevators_avg,ft_app_entrances_avg,ft_app_livingapartments_avg,ft_app_livingarea_avg,ft_app_nonlivingapartments_avg,ft_app_nonlivingarea_avg,ft_app_floorsmax_avg,ft_app_floorsmin_avg,ft_app_landarea_avg,ft_app_years_beginexpluatation_avg,ft_app_years_build_avg,ft_app_cnt_children,ft_app_cnt_fam_members,ft_app_days_birth,ft_app_days_id_publish,ft_app_days_employed,ft_app_own_car_age,ft_app_days_last_phone_change,ft_app_days_registration,ft_app_def_30_cnt_social_circle,ft_app_def_60_cnt_social_circle,ft_app_obs_30_cnt_social_circle,ft_app_ext_source_1,ft_app_ext_source_2,ft_app_ext_source_3,ft_app_region_population_relative,ft_app_region_rating_client,ft_app_totalarea_mode
0,100002,0,20.112329,0.121978,2.007889,16.461104,1.158397,0.0,202500.0,202500.0,0.067329,1.780220,0,cat_M,cat_WEDNESDAY,cat_10,cat_1,cat_1,cat_0,cat_1,cat_0,cat_1,cat_1,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_N,cat_Y,cat_Laborers,cat_Business Entity Type 3,cat_Working,cat_Cash loans,cat_Secondary / secondary special,cat_Single / not married,cat_House / apartment,cat_Unaccompanied,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_block of flats,cat_No,cat_reg oper account,"cat_Stone, brick",24700.5,406597.5,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0369,0.0143,0.00,0.0690,0.0202,0.0190,0.0000,0.0000,0.0833,0.1250,0.0369,0.9722,0.6192,0,1.0,-9461,-2120,-637.0,NaN,-1134.0,-3648.0,2.0,2.0,2.0,0.083037,0.262949,0.139376,0.018801,2,0.0149
1,100003,1,45.134247,0.132217,4.790750,36.234085,1.145199,0.0,270000.0,135000.0,0.070862,0.696970,0,cat_F,cat_MONDAY,cat_11,cat_1,cat_1,cat_0,cat_1,cat_0,cat_1,cat_1,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_N,cat_N,cat_Core staff,cat_School,cat_State servant,cat_Cash loans,cat_Higher education,cat_Married,cat_House / apartment,cat_Family,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_block of flats,cat_No,cat_reg oper account,cat_Block,35698.5,1293502.5,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0959,0.0529,0.0605,0.08,0.0345,0.0773,0.0549,0.0039,0.0098,0.2917,0.3333,0.0130,0.9851,0.7960,0,2.0,-16765,-291,-1188.0,NaN,-828.0,-1186.0,0.0,0.0,1.0,0.311267,0.622246,NaN,0.003541,1,0.0714
2,100004,1,45.246575,0.100000,2.000000,20.000000,1.000000,0.0,67500.0,67500.0,0.011814,3.622222,0,cat_M,cat_MONDAY,cat_9,cat_1,cat_1,cat_0,cat_1,cat_1,cat_1,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_0,cat_Y,cat_Y,cat_Laborers,cat_Government,cat_Working,cat_Revolving loans,cat_